In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
!pip install -r ./yolov5/requirements.txt

In [ ]:
!rm -r ./temp
!rm -r ./temp2
!rm -r ./crop
!rm -r ./1
!rm -r ./2
!rm -r ./3

!mkdir ./temp
!mkdir ./1
!mkdir ./2
!mkdir ./3

In [ ]:
!cp -R ./test/* ./temp

In [ ]:
import os

subdirs = os.listdir('./temp')
subdirs

In [ ]:
for i in subdirs:
    os.rename('./temp/'+i, './temp/'+i[:-4]+'__'+i[-4:])


In [ ]:
!python ./yolov5/detect.py --weights yolov5x6.pt  --conf 0.5 --save-crop --source ./temp/ --project ./crop/ --name Cygnus --exist-ok


In [ ]:
!mkdir ./temp2
!cp -R ./crop/Cygnus/crops/bird/* ./temp2

In [ ]:
import os

import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [15]:
class_names = ['Cygnus_bewickii', 'Cygnus_cygnus', 'Cygnus_olor', 'other_bird']

model = tf.keras.models.load_model("model_mobilenet_stage2.h5", compile=False)

In [16]:
BATCH_SIZE = 1
IMG_SIZE = (224, 224)

datagen = tf.keras.preprocessing.image.ImageDataGenerator()
filter_dataset = datagen.flow_from_directory('./temp2/',
                                             target_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE,
                                             shuffle=False,
                                             classes=[''])
prediction = model.predict(filter_dataset, verbose=1)
fn = filter_dataset.filenames

Found 14 images belonging to 1 classes.
14/14 [==============================] - 2s 49ms/step


In [17]:
D = {}

cnt = 0
fn = filter_dataset.filenames
prediction1 = np.argmax(prediction[:, :], axis=1)
prediction2 = np.argmax(prediction[:, :-1], axis=1)

for p1, p2 in zip(prediction1, prediction2):
    D[fn[cnt]] = {'prediction_all': p1, 'prediction_no_other': p2}
    cnt += 1

D = pd.DataFrame(D)
D = D.T.reset_index()
D.columns = ['file_name', 'prediction_all', 'prediction_no_other']
D['crop_num'] = D['file_name'].apply(lambda x: x.split('__')[1]).values
D['file_name'] = D['file_name'].apply(lambda x: x.split('__')[0]+'.'+x.split('__')[1].split('.')[-1]).values


In [18]:
D

,file_name,prediction_all,prediction_no_other,crop_num
0,FileB.jpg,0,0,1.jpg
1,FileB.jpg,0,0,2.jpg
2,FileB.jpg,3,2,3.jpg
3,figC.jpg,2,2,1 (1).jpg
4,figC.jpg,2,2,1 (2).jpg
5,figC.jpg,2,2,1 (3).jpg
6,figD.png,2,2,1.png
7,figE.png,2,2,1.png
8,figF.png,1,1,1.png
9,figG.png,2,2,1.png


In [21]:
# E = D.groupby(['file_name'])['prediction_no_other'].mean().astype(int).reset_index()
# E.columns = ['name', 'class']  # comment!!
# E

results = {}

for file_name in D['file_name'].unique():
    sub = D[D.file_name == file_name]
    try:
        #TRY to delete other birds, when vote
        res = sub[sub.prediction_all != 3]['prediction_all'].value_counts().index.values[0]
    except Exception as e:
        # if all crops are "other birds", try softmax without "other bird"
        res = sub['prediction_no_other'].value_counts().index.values.values[0]

    results[file_name] = [class_names[res]]

submit = pd.DataFrame(results).T.reset_index()
submit.columns = ['name', 'class']
SMAP = {'Cygnus_olor': 3, 'Cygnus_bewickii': 1, 'Cygnus_cygnus': 2}
submit['class'] = submit['class'].map(SMAP)

E = submit.copy()
E

,name,class
0,FileB.jpg,1
1,figC.jpg,3
2,figD.png,3
3,figE.png,3
4,figF.png,2
5,figG.png,3
6,figH.png,1
7,figI.png,1
8,fileA.jpg,1


In [ ]:
# E['class']=E['class']+1 # Do not need it
# E

In [22]:
E.to_csv(f"submission_v2.csv", sep=';', index = False)

In [ ]:
import shutil

for i in E.index:
    shutil.copyfile('./test/'+E['name'].loc[i], './'+E['class'].loc[i].astype(str)+'/'+E['name'].loc[i])
    